In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001

Capacitance = 2.
Inductance = 2.
R = 1

solver = "gp"
time_limit=1000

# Ciropt problem

In [3]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)

res, sol = problem.solve(verbose=True, solver=solver, time_limit=time_limit)[:2]
vars = problem.vars

dim_G=6, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 1000
Set parameter FeasibilityTol to value 1e-09
Set parameter Method to value 0
Set parameter PoolSearchMode to value 1
Set parameter Heuristics to value 0.001
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 89 columns and 51 nonzeros
Model fingerprint: 0xe38c1fe4
Model has 68 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-06, 1e+00]
  QLMatrix range   [5e-05, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Presolve removed 26 rows and 19 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 25 rows and 19 columns
Presolve time:

In [ ]:
res

# PEP verification

In [ ]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
d = res["d"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [ ]:
problem = PEPit.PEP()
func = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   # Strong convexity param.
                    L=L_smooth) 

x_P_star = func.stationary_point()
g_star, f_star = func.oracle(x_P_star)

x_C_1 = problem.set_initial_point()
y_L_1 = problem.set_initial_point()


x_P_1, g_1, f_1 = proximal_step((R * y_L_1 + x_C_1), func, R)

y_L_1p5 = y_L_1 + (alpha * h / Inductance) * (x_C_1 - x_P_1) 
x_C_1p5 = x_C_1 - (alpha * h / Capacitance) * g_1 
x_P_1p5, g_1p5, f_1p5 = proximal_step((R * y_L_1p5 + x_C_1p5), func, R)

y_L_2 = y_L_1 + (beta * h / Inductance) * (x_C_1 - x_P_1) + \
                ((1 - beta) * h / Inductance) * (x_C_1p5 - x_P_1p5)
x_C_2 = x_C_1 - (beta * h / Capacitance) * g_1 - ((1 - beta) * h / Capacitance) * g_1p5  
x_P_2, g_2, f_2 = proximal_step((R * y_L_2 + x_C_2), func, R)

# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (x_C_1 - x_P_star)**2 + (Inductance/2) * (y_L_1 - g_star) ** 2
E_2 = (Capacitance/2) * (x_C_2 - x_P_star)**2 + (Inductance/2) * (y_L_2 - g_star) ** 2
Delta_1 = d * R * (g_1 - y_L_1)**2 + b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")